# Extract Time Series Sentinel-2 data
## from Digital Earth Australia (DEA) via STAC

In [1]:
!python --version

Python 3.8.13


In [2]:
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import folium
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

import pystac_client
import odc.stac
# odc-stac library downloads DEA datasets stored in AWS
# when external to AWS (like outside DEA sandbox), AWS signed requests must be disabled
os.environ['AWS_NO_SIGN_REQUEST'] = 'YES'

from datacube.utils.geometry import point, box, CRS, Geometry, Coordinate, BoundingBox, GeoBox
from datacube.model import GridSpec
from affine import Affine

# My helper class
from helperfunctions import begin_timer, end_timer, saveDataset, loadDataset
from dea_helperfunctions import DEA_HelperFunctions

from dea_tools.plotting import rgb
from dea_tools.bandindices import calculate_indices

# Set some configurations for displaying tables nicely
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_rows", None)

### Constants

In [3]:
# configure helper functions
dea_fns = DEA_HelperFunctions()
basepath = "../datasets/Sentinel2withIndices_"
fileextn = ".pkl"

filespecifier = "PortGawlerBeachSA_2022_Mar"

# Survey Area origin point
survey_origin = [138.43196647747274, -34.62929501472954 ] # PortGawlerBeach, SA
timebands = [["2022-03-01","2022-03-05"]] # Mar 2022

# Coordinate Reference Systems (CRS)
# !! ENSURE YOU USING CONSISTENT CRS WHEN PLOTTING AREAS OR PERFORMING GEOMETRIC OPERATIONS.
epsg4326 = 'epsg:4326'            # EPSG:4326 | WGS84 latitude-longitude CRS | in Degrees of Latitude and Longitude
epsg3577 = 'epsg:3577'            # EPSG:3577 | GDA94 / Australian Albers projection | in Metres from CRS Centroid

# Survey Area (Region of Interest)
resolution_int = 10 # where each pixel is of 10m x 10m resolution
resolution = (-resolution_int,resolution_int) # where each pixel is of 10m x 10m resolution
survey_tilesize = (5120,5120)     # size of each tile in metres
survey_tiles = (2,2)              # number of horizontal and vertical tiles change to 10,5 for final run

survey_tilesize_pixels = tuple(int(ti/resolution_int) for ti in survey_tilesize)
print(survey_tilesize_pixels)

# Satellite datasets
collections = ["s2a_ard_granule","s2b_ard_granule"] # Sentinel-2A and 2B MSI Definitive ARD - NBART and Pixel Quality
bands = ("nbart_blue", "nbart_green", "nbart_red", "nbart_nir_1") # Satellite Bands

# The following band indices are added to the datasets
#        'NDVI' (Normalised Difference Vegetation Index, Rouse 1973)
#        'NDWI' (Normalised Difference Water Index, McFeeters 1996)
#        'kNDVI' (Non-linear Normalised Difference Vegetation Index, Camps-Valls et al. 2021)
#         Note: kNDVI is more resistant to saturation, bias, and complex phenological cycles
#               and shows enhanced robustness to noise and stability across spatial and temporal scales.
additional_bands = ['NDVI','kNDVI','NDWI']

(512, 512)


### Define and Display Survey Area Grid Tiles

In [4]:
# Subdivide survey area into tiles
surveyarea_polygons = dea_fns.calc_surveyarea_polygons( origin=survey_origin,
                                                       tileresolution=survey_tilesize,
                                                       numtiles=survey_tiles)
# Add polygons to geodataframe
gdf_surveyarea = gpd.GeoDataFrame(columns=["tile", "geometry"], crs=epsg4326)

tilenum = 0
for surveyarea_polygon in surveyarea_polygons:
    tilenum +=1
    gdf_surveyarea.loc[tilenum] = (tilenum,surveyarea_polygon)

# Save data set to pickle file
saveDataset(basepath + filespecifier + "_tilepolygons" + fileextn, gdf_surveyarea)

# display Tile areas
gdf_surveyarea.tail(5)

,tile,geometry
1,1,"POLYGON ((138.43197 -34.62930, 138.43197 -34.58325, 138.48793 -34.58325, 138.48793 -34.62930, 138.43197 -34.62930))"
2,2,"POLYGON ((138.48793 -34.62930, 138.48793 -34.58325, 138.54388 -34.58325, 138.54388 -34.62930, 138.48793 -34.62930))"
3,3,"POLYGON ((138.43197 -34.58325, 138.43197 -34.53720, 138.48793 -34.53720, 138.48793 -34.58325, 138.43197 -34.58325))"
4,4,"POLYGON ((138.48793 -34.58325, 138.48793 -34.53720, 138.54388 -34.53720, 138.54388 -34.58325, 138.48793 -34.58325))"


In [5]:
map_zoom_level = 9
map_centroid = survey_origin.reverse() # swap lat and long

m = gdf_surveyarea.explore(
     column="tile", # make choropleth based on Commodity description
     tooltip=["tile"], popup=True, # show all values in popup (on click)
     tiles="OpenStreetMap", # use "CartoDB positron" tiles
     cmap="Wistia", # use "Set1" matplotlib colormap
     legend=False,
     location=map_centroid,
     zoom_start = map_zoom_level,
     name='Survey Area Tiles') # use black outline

folium.TileLayer('CartoDB positron', control=True).add_to(m)  # use folium to add alternative tiles
folium.LayerControl().add_to(m)  # use folium to add layer control

m  # show map

### Open Catalog & Extract images from Survey area

In [6]:
catalog = pystac_client.Client.open('https://explorer.sandbox.dea.ga.gov.au/stac')

stime = begin_timer()

# Iterate through each GeoDataFrame polygon
# Extract bands for each polygon for each time period specified

surveyarea_ds = []

for t in timebands:
    i=0
    datem = datetime. datetime. strptime(t[0], "%Y-%m-%d")
    y = datem.year # year
    m = datem.month # month
    for p in surveyarea_polygons:
        i+=1
        bb_4326 = dea_fns.convert_poly2bbox(p)
        b_4326 = dea_fns.convert_poly2box(p)
        b = b_4326.to_crs(epsg3577)
        bb = b.boundingbox
        
        query = catalog.search( bbox=bb_4326, collections=collections, datetime=f"{t[0]}/{t[1]}" )
        items = list(query.get_items()) # Search the STAC catalog
        
        print(f"Time band: {t[0]} to {t[1]} | Tile #: {i} | Found: {len(items):d} datasets")
        print(bb_4326)
        print("---------------------------------------------------------------")
        
        if len(items) > 0:
            surveyarea_affine = Affine(resolution[1], 0.0, bb.left, 0.0, resolution[0], bb.bottom)
            ds = odc.stac.load( items,
                                bands=bands,
                                geobox=GeoBox(survey_tilesize_pixels[0], # width in pixels
                                              survey_tilesize_pixels[1], # height in pixels
                                              surveyarea_affine,
                                              epsg3577), # Output Coordinate Reference System (CRS)
                                groupby="solar_day")

            calculate_indices(ds=ds, index=additional_bands, collection='ga_s2_1', inplace=True)
            surveyarea_ds.append([t, y, m, len(items), i, p, ds])      

end_timer(stime)

# Save data set to pickle file
saveDataset(basepath + filespecifier + fileextn, surveyarea_ds)

[ Begin timer at Mon May 16 11:27:00 2022 ]
Time band: 2022-03-01 to 2022-03-05 | Tile #: 1 | Found: 2 datasets
BoundingBox(left=138.43196647747274, bottom=-34.62929501472954, right=138.48792503199041, top=-34.583249748506496)
---------------------------------------------------------------
Time band: 2022-03-01 to 2022-03-05 | Tile #: 2 | Found: 2 datasets
BoundingBox(left=138.48792503199041, bottom=-34.62929501472954, right=138.54388358650806, top=-34.583249748506496)
---------------------------------------------------------------
Time band: 2022-03-01 to 2022-03-05 | Tile #: 3 | Found: 2 datasets
BoundingBox(left=138.43196647747274, bottom=-34.583249748506496, right=138.48792503199041, top=-34.537204482283464)
---------------------------------------------------------------
Time band: 2022-03-01 to 2022-03-05 | Tile #: 4 | Found: 2 datasets
BoundingBox(left=138.48792503199041, bottom=-34.583249748506496, right=138.54388358650806, top=-34.537204482283464)
------------------------------

In [7]:
print(ds)        # show data structure of last extract

<xarray.Dataset>
Dimensions:      (time: 2, y: 512, x: 512)
Coordinates:
  * time         (time) datetime64[ns] 2022-03-02T00:46:52.624724 2022-03-05T...
  * y            (y) float64 -3.791e+06 -3.791e+06 ... -3.796e+06 -3.796e+06
  * x            (x) float64 5.927e+05 5.927e+05 ... 5.978e+05 5.978e+05
    spatial_ref  int32 3577
Data variables:
    nbart_blue   (time, y, x) uint16 861 836 821 823 819 ... 454 443 421 403 450
    nbart_green  (time, y, x) uint16 1079 1027 1018 998 986 ... 485 469 500 511
    nbart_red    (time, y, x) uint16 1410 1292 1274 1271 ... 628 539 538 609
    nbart_nir_1  (time, y, x) uint16 2329 2251 2276 2207 ... 1160 1124 1101 1118
    NDVI         (time, y, x) float64 0.2458 0.2707 0.2823 ... 0.3435 0.2947
    kNDVI        (time, y, x) float64 0.06034 0.07313 0.0795 ... 0.1174 0.08665
    NDWI         (time, y, x) float64 -0.3668 -0.3734 ... -0.3754 -0.3726
Attributes:
    crs:           epsg:3577
    grid_mapping:  spatial_ref
